# Tensors in torch

In [2]:
import torch

## 1. Creating tensors 

### Basics

Tensors can be created using specific commands:

In [8]:
torch.empty(3, 3)

(tensor([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]), tensor([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]))

In [9]:
torch.zeros(3, 3)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [14]:
torch.rand(1,3,1, dtype=torch.double)

tensor([[[0.8695],
         [0.0413],
         [0.1245]]], dtype=torch.float64)

Or directly from data:

In [19]:
torch.Tensor([[1,2,3]])

tensor([[1., 2., 3.]])

### `torch.Tensor` attributes

Each `torch.Tensor` object has three attributtes:
* `torch.dtype`
* `torch.device`
* `torch.layout`

#### dtype

A `torch.dtype` is an object that represents the type of a `torch.Tensor`. There are **nine** different dtypes in torch:

https://pytorch.org/docs/stable/tensor_attributes.html

https://pytorch.org/docs/stable/tensors.html

https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html